In [ ]:
# 실험 코드: 동일 회로, 피드백 여부에 따른 정보 흐름 비교

import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from scipy.spatial.distance import cosine
from scipy.stats import entropy as shannon_entropy
import matplotlib.pyplot as plt

# 시뮬레이터 설정
sim = Aer.get_backend('qasm_simulator')
shots = 1024
n = 3  # 큐비트 수

# 회로 실행 함수
def run_circuit(U):
    qc = QuantumCircuit(n, n)
    for i in range(n):
        qc.ry(U[i], i)
    qc.barrier()
    qc.measure(range(n), range(n))
    result = execute(qc, sim, shots=shots).result()
    return result.get_counts()

# 피드백 업데이트 함수 (출력에 따라 각도 조정)
def update_U(U, counts, lr=0.05):
    total = sum(counts.values())
    avg_bits = np.zeros(n)
    for bitstring, count in counts.items():
        bits = np.array([int(b) for b in bitstring[::-1]])  # Qiskit은 역순
        avg_bits += bits * (count / total)
    return U + lr * (avg_bits - 0.5) * np.pi  # 0.5는 균형 기준점

# 출력 분포 정규화
def normalize_counts(counts):
    total = sum(counts.values())
    return {k: v / total for k, v in counts.items()}

# 유사도 계산 (cosine)
def compare_cosine(counts1, counts2):
    keys = set(counts1) | set(counts2)
    v1 = np.array([counts1.get(k, 0) for k in keys])
    v2 = np.array([counts2.get(k, 0) for k in keys])
    return 1 - cosine(v1, v2)

# 분포 집중도 (엔트로피)
def compute_entropy(counts):
    norm = np.array(list(normalize_counts(counts).values()))
    return shannon_entropy(norm, base=2)

# 실험 반복 설정
steps = 30
U0 = np.random.uniform(0, np.pi, size=n)

# 무피드백 조건
U_no_fb = U0.copy()
counts_no_fb_list = []
for _ in range(steps):
    counts = run_circuit(U_no_fb)
    counts_no_fb_list.append(counts)

# 피드백 조건
U_fb = U0.copy()
counts_fb_list = []
for _ in range(steps):
    counts = run_circuit(U_fb)
    counts_fb_list.append(counts)
    U_fb = update_U(U_fb, counts)

# 결과 비교: 엔트로피 변화 시각화
def plot_entropy(counts_list, label):
    entropies = [compute_entropy(c) for c in counts_list]
    plt.plot(entropies, label=label)

plt.figure(figsize=(10, 5))
plot_entropy(counts_no_fb_list, "No Feedback")
plot_entropy(counts_fb_list, "With Feedback")
plt.xlabel("Step")
plt.ylabel("Entropy")
plt.legend()
plt.title("Shannon Entropy Over Time (3-Qubit RY)")
plt.grid(True)
plt.show()

# 유사도 변화도 출력 (원하면 여기에 추가 가능)
